In [1]:
import numpy as np
import pandas as pd
import QUANTAXIS as qa
import datetime as dt
from functools import reduce

QUANTAXIS>> start QUANTAXIS
QUANTAXIS>> failed to import pyecharts_snapshot
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/lml/utils.py", line 43, in do_import
    plugin_module = __import__(plugin_module_name)
ModuleNotFoundError: No module named 'pyecharts_snapshot'
QUANTAXIS>> Expand macros in /usr/local/lib/python3.7/site-packages/macropy/core/hquotes.py
QUANTAXIS>> Finding macros in 'macropy.core.hquotes'
QUANTAXIS>> Importing macros from 'macropy.core.quotes' into 'macropy.core.hquotes'
QUANTAXIS>> Expand macros in /usr/local/lib/python3.7/site-packages/macropy/core/quotes.py
QUANTAXIS>> Finding macros in 'macropy.core.quotes'
QUANTAXIS>> Expand macros in /usr/local/lib/python3.7/site-packages/macropy/core/failure.py
QUANTAXIS>> Finding macros in 'macropy.core.failure'
QUANTAXIS>> Importing macros from 'macropy.core.hquotes' into 'macropy.core.failure'
QUANTAXIS>> Expand macros in /usr/local/lib/python3.7/site-packages/cffi/api.py
QUANTAXIS>> Fi

no display found. Using non-interactive Agg backend
if you use ssh, you can use ssh with -X parmas to avoid this issue


QUANTAXIS>> Expand macros in /usr/local/lib/python3.7/site-packages/scipy/__config__.py
QUANTAXIS>> Finding macros in 'scipy.__config__'
QUANTAXIS>> Welcome to QUANTAXIS, the Version is 1.0.68
QUANTAXIS>>  
 ```````````````````````````````````````````````````````````````````````````````````````````````````````````````````````` 
  ``########`````##````````##``````````##`````````####````````##```##########````````#``````##``````###```##`````######`` 
  `##``````## ```##````````##`````````####````````##`##```````##```````##```````````###``````##````##`````##```##`````##` 
  ##````````##```##````````##````````##`##````````##``##``````##```````##``````````####```````#```##``````##```##``````## 
  ##````````##```##````````##```````##```##```````##```##`````##```````##`````````##`##```````##`##```````##````##``````` 
  ##````````##```##````````##``````##`````##``````##````##````##```````##````````##``###```````###````````##`````##`````` 
  ##````````##```##````````##``````##``````##`````##```

In [42]:
def go_up(data, N=5, P=3, M=0.1):
    ind = (data.close > data.close.shift(1)) * 1
    checked = check(ind, -N)
    lift = data.high.rolling(N).max().shift(-N-1) / data.open.shift(-1)
    flat = ((data.close == data.open) & (data.close == data.high)) * 1    
    return pd.DataFrame({'GOUP': (~(flat > 0) & ~(flat.shift(-1) > 0) & (checked >= P) & (lift > (1+M))) * 1})


def check(signal, days):
    return reduce(lambda x, y: x + y, [signal.shift(x).fillna(0)
                                       for x in (range(days) if days >= 0 else range(-1, days-1, -1))])


In [ ]:
today = dt.datetime.today()
# today = dt.datetime(2018, 7, 6)
today_str = today.strftime('%Y-%m-%d')

# start_date = today - dt.timedelta(days=365)

stocks = qa.QA_fetch_stock_list_adv()
stock_list = stocks.code.tolist()
for stock in stock_list[:10]:
    print('handling %s' % stock)
    try:
        candles = qa.QA_fetch_stock_day_adv(stock, start='2014-01-01', end=today_str).to_qfq()
    except:
        print('data error during {}'.format(stock))
        continue

In [4]:
candles = qa.QA_fetch_stock_day_adv('000001', start='2014-01-01', end='2018-08-18').to_qfq()

In [43]:
ca = candles.add_func(go_up)

In [44]:
dd = candles.data
dd['rh'] = candles.data.high.rolling(5).max().shift(-6)
dd['goup'] = ca
dd['next'] = candles.data.open.shift(-1)
dd['upup'] = dd['rh'] / dd['next'] > 1.1
dd

,,open,high,low,close,volume,amount,preclose,adj,rh,goup,next,upup
date,code,,,,,,,,,,,,
2014-01-02,000001,6.556834,6.654213,6.518965,6.616344,9.055756e+05,5.962237e+08,NaN,0.540993,6.491915,0,6.573064,False
2014-01-03,000001,6.573064,6.578474,6.372897,6.454046,1.018708e+06,6.566313e+08,6.616344,0.540993,6.486505,0,6.432406,False
2014-01-06,000001,6.432406,6.491915,6.221419,6.313388,1.076018e+06,6.792804e+08,6.454046,0.540993,6.486505,0,6.237649,False
2014-01-07,000001,6.237649,6.362077,6.226829,6.291748,6.255294e+05,3.939776e+08,6.313388,0.540993,6.486505,0,6.297158,False
2014-01-08,000001,6.297158,6.464866,6.237649,6.362077,8.461626e+05,5.384362e+08,6.291748,0.540993,6.464866,0,6.324207,False
2014-01-09,000001,6.324207,6.486505,6.302568,6.394537,8.974572e+05,5.768705e+08,6.362077,0.540993,6.443226,0,6.372897,False
2014-01-10,000001,6.372897,6.464866,6.307978,6.394537,7.046247e+05,4.504877e+08,6.394537,0.540993,6.383717,0,6.383717,False
2014-01-13,000001,6.383717,6.443226,6.216009,6.275518,8.851779e+05,5.593752e+08,6.394537,0.540993,6.383717,0,6.259288,False
2014-01-14,000001,6.259288,6.372897,6.194369,6.340437,6.699293e+05,4.207153e+08,6.275518,0.540993,6.416176,0,6.329617,False


In [45]:
dd.loc[dd.goup > 0, :].shape

(0, 12)

In [49]:
 
dd['skdj.k.9.3'], dd['skdj.d.9.3'] = candles.add_func(qa.QA_indicator_SKDJ, N=9, M=3).loc[:, ['SKDJ_K', 'SKDJ_D']]
dd

,,open,high,low,close,volume,amount,preclose,adj,rh,goup,next,upup,skdj.k.9.3,skdj.d.9.3
date,code,,,,,,,,,,,,,,
2014-01-02,000001,6.556834,6.654213,6.518965,6.616344,9.055756e+05,5.962237e+08,NaN,0.540993,6.491915,0,6.573064,False,SKDJ_K,SKDJ_D
2014-01-03,000001,6.573064,6.578474,6.372897,6.454046,1.018708e+06,6.566313e+08,6.616344,0.540993,6.486505,0,6.432406,False,SKDJ_K,SKDJ_D
2014-01-06,000001,6.432406,6.491915,6.221419,6.313388,1.076018e+06,6.792804e+08,6.454046,0.540993,6.486505,0,6.237649,False,SKDJ_K,SKDJ_D
2014-01-07,000001,6.237649,6.362077,6.226829,6.291748,6.255294e+05,3.939776e+08,6.313388,0.540993,6.486505,0,6.297158,False,SKDJ_K,SKDJ_D
2014-01-08,000001,6.297158,6.464866,6.237649,6.362077,8.461626e+05,5.384362e+08,6.291748,0.540993,6.464866,0,6.324207,False,SKDJ_K,SKDJ_D
2014-01-09,000001,6.324207,6.486505,6.302568,6.394537,8.974572e+05,5.768705e+08,6.362077,0.540993,6.443226,0,6.372897,False,SKDJ_K,SKDJ_D
2014-01-10,000001,6.372897,6.464866,6.307978,6.394537,7.046247e+05,4.504877e+08,6.394537,0.540993,6.383717,0,6.383717,False,SKDJ_K,SKDJ_D
2014-01-13,000001,6.383717,6.443226,6.216009,6.275518,8.851779e+05,5.593752e+08,6.394537,0.540993,6.383717,0,6.259288,False,SKDJ_K,SKDJ_D
2014-01-14,000001,6.259288,6.372897,6.194369,6.340437,6.699293e+05,4.207153e+08,6.275518,0.540993,6.416176,0,6.329617,False,SKDJ_K,SKDJ_D
